In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
df = pd.read_csv('/kaggle/input/bitcoin-price-prediction/btcNewsToPrice2.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.fillna("",inplace=True)

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.plot(df['price'])

In [ ]:
df.tail(100)

In [ ]:
df[df['date'] <= '2021-04-30'].tail()

In [ ]:
train = df[df['date'] <= '2021-04-30']
test = df[df['date'] > '2021-04-30']

In [ ]:
train.shape, test.shape

In [ ]:
train.iloc[0,1:37]

In [ ]:
headlines_train = []
for row in range(0,len(train.index)):
    headlines_train.append(' '.join(str(x) for x in train.iloc[row,1:37]))

In [ ]:
headlines_train[0:5]

In [ ]:
headlines_test= []
for row in range(0,len(test.index)):
    headlines_test.append(' '.join(str(x) for x in test.iloc[row,1:37]))

In [ ]:
for i in range(len(headlines_train)):
    headlines_train[i] = " ".join(headlines_train[i].split())

In [ ]:
headlines_train[0:5]

In [ ]:
for i in range(len(headlines_test)):
    headlines_test[i] = " ".join(headlines_test[i].split())

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
max_length = []

for i in range(len(headlines_train)):
    max_length.append(len(headlines_train[i].split()))

In [ ]:
plt.plot(max_length)

In [ ]:
countvector=CountVectorizer(ngram_range=(2,2),max_features=200)
traindataset=countvector.fit_transform(headlines_train)

randomclassifier=RandomForestRegressor(n_estimators=200,n_jobs=-1)
randomclassifier.fit(traindataset,train['price'])

In [ ]:
test_dataset = countvector.transform(headlines_test)
predictions = randomclassifier.predict(test_dataset)

In [ ]:
np.sqrt(metrics.mean_squared_error(test['price'], predictions))

In [ ]:
test['price'].values

In [ ]:
plt.plot(test['price'].values)

In [ ]:
predictions

In [ ]:
plt.plot(predictions)

In [ ]:
%%time

max_features_num = [200,300,400,500,600,700]
ngram = [1,2,3]
for i in max_features_num:
    for j in ngram:
        countvector=CountVectorizer(ngram_range=(j,j),max_features=i)
        traindataset=countvector.fit_transform(headlines_train)
        test_dataset = countvector.transform(headlines_test)

        xgb = XGBRegressor(random_state =1)
        xgb.fit(pd.DataFrame(traindataset.todense(), columns=countvector.get_feature_names()),train['price'])
        predictions = xgb.predict(pd.DataFrame(test_dataset.todense(), columns=countvector.get_feature_names()))
        score=np.sqrt(metrics.mean_squared_error(test['price'],predictions))
        print('max number of features used : {}'.format(i))
        print('ngram_range ({},{})'.format(j,j))
        print(score)
        print(test['price'].values)
        print(predictions)
        print("=============================================")